### Code

In [ ]:
j_lvector       = np.linspace (minj_l, maxj_l, num = testpointsj_l)
j_gvector       = np.linspace (minj_g, maxj_g, num = testpointsj_g)
j_lmapstability = np.zeros ((testpointsbeta,testpointsj_l)) #(rows,columns)

k = 0
for beta in betavector:
    print ("  >>> β = ", np.rad2deg(beta), "deg")
    j = 0 #for each element of j2vector
    for j_g in j_gvector: #iterative process for j2
        i = 0 #for each element of j1vector
        for j_l in j_lvector: #iterative process for j1
            var4 = var4_0
            var1eq = brenth (equilibrium1, lima, limb)
            
            # parameters used in boundary conditions
            var1_0 = var1eq
            var2_0 = var2 (var1eq)
            var3_0 = var3 (var1eq)
            ref = np.array([var1_0, var2_0, var3_0, var4_0])
            
            Aeval = np.asarray (Amat (var1_0, var2_0, var3_0, var4_0))
            Beval = np.asarray (Bmat (var1_0, var2_0, var3_0, var4_0))
            Ceval = np.asarray (Cmat_lin (var1_0, var2_0, var3_0, var4_0))

            Acomplex   = Aeval.dot (1j)
            Bcomplex   = Beval.dot (1j)
            Ccomplex   = Ceval.dot (1j)
            
            #criterion for well-posedness
            m2, _ = eig ( (-wavenumber_fourier*Bcomplex - Ceval), b =  -Acomplex, overwrite_a = True, overwrite_b = True, check_finite = True) 
            
            listreal2   = []
            listimag2   = []
            
            for l2 in range (len (m2)):
                realpart = m2 [l2].real                
                imagpart = m2 [l2].imag
                
                listimag2.append (imagpart)
                listreal2.append (realpart)
            
            if all ([ v > 0 for v in listimag2]) :
                i += 1
            else: 
                j_lmapstability [k, j] = j_l
                break
        j += 1
    k += 1

### Plot neutral stability curves

In [ ]:
x = j_gvector
fig, ax = plt.subplots ()

for i in range(testpointsbeta):
    ax.loglog (x,
               j_lmapstability [i, :],
               liststyles [i],
               color = 'k' ,
               label = r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')

for i in range (testpointsj_l_cases):
    ax.scatter (j_gcases [i], 
                j_lcases [i],
                s = areatestpoints, 
                color = listcolor [0], 
                marker = listmarkers[i], 
                edgecolors = listcolor [0], 
                linewidths = 1, 
                alpha = alphascatter)
    ax.set_xscale ('symlog')

    #ax.set_xscale ('symlog')
    #ax.set_yscale ('symlog')
    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s')

# Savitzky-Golay Filter
# for i in range (testpointsbeta): #scipy.signal.savgol_filter: window size 51, polynomial order 3  *******63, 1
#     ax.loglog (x, scipy.signal.savgol_filter(j1mapstability [i,:], 93, 1), liststyles [i], color = 'k', label = '$beta$' + ' = '+str("%.2f" %betavectordeg [i]) + ' deg')

#plt.title('Well-posedness limit for horizontal and inclined pipes')

plt.ylim ((minj_l, maxj_l))
plt.xlim ((minj_g, maxj_g))

#tick_params(labeltop=False, labelright=True)
plt.grid (True, which = "both")
plt.rcParams ['figure.figsize'] = [12, 8]
leg = ax.legend (loc = 'best', fontsize = 14, shadow = True);

matplotlib.rc ('xtick', labelsize = 18)     
matplotlib.rc ('ytick', labelsize = 18)

ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$',
            fontsize = 18)
ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$',
            fontsize = 18)

plt.show ()
# plt.savefig('figures/stability_map_1.pdf',
                # optimize = True,
                # transparent = True,  
                # dpi = 1000)

### Neutral curves + well-posedness + verification

In [ ]:
x = j_gvector

i = 0
for i in range(testpointsbeta):
    fig, ax    = plt.subplots ()
    criteria   = ["VKH limit", "IKH limit"]
    
    # neutral stability
    ax.loglog (x,
               j_lmapwellposedness [i, :],
               liststyles [2],
               color = listcolor [0],
               label = criteria [1] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')
    ax.loglog (x, j_lmapstability [i, :], liststyles [i], color = 'k', label = criteria [0] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')
    
    # Savitzky-Golay Filter ****83, 1
    # ax.loglog (x,
    #            scipy.signal.savgol_filter(j_lmapstability [i,:], 93, 1), #93, 1
    #            liststyles [0], color = listcolor [0],
    #            label = criteria [0] + ' , ' + '$β$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' [°]')
    
    # # line of constant alpha
    # ax.loglog (x,
    #            j_llinemap [i, :],
    #            liststyles [1],
    #            color = listcolor [0],
    #            label =  r'$\alpha_l$ = 0.5 [-]')
  # Fill
    # ax.fill_between(x,
    #                 scipy.signal.savgol_filter(j_lmapstability [i,:], 9, 1),
    #                 j_lmapwellposedness [i, :],
    #                 where = j_lmapwellposedness [i, :] >= scipy.signal.savgol_filter(j_lmapstability [i,:], 113, 1),
    #                 facecolor = 'silver',
    #                 interpolate = True)    
    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s') 
    
    #Scatter
#     ax.plot(j_gtest2 [i, :], j_ltest2 [i, :], 's', color = 'black')

    for i in range (testpointsj_l_cases):
        ax.scatter (j_gcases [i], j_lcases [i], s = areatestpoints, color = listcolor [4], marker = listmarkers[i], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
        ax.set_xscale ('symlog')

    plt.ylim ((minj_l, maxj_l))
    plt.xlim ((minj_g, maxj_g))

    #plt.title('Well-posedness limit for horizontal and inclined pipes')
    #tick_params(labeltop=False, labelright=True)
    plt.grid (True, which = "both")
    plt.rcParams ['figure.figsize'] = [12, 8]
    leg = ax.legend (loc = 'upper right', frameon = True, fontsize = 14, shadow = True);

    matplotlib.rc ('xtick', labelsize = 18)     
    matplotlib.rc ('ytick', labelsize = 18)

    ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

    ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$',
            fontsize = 18)
    ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$',
            fontsize = 18)

    #>>>>>>>>>>VALIDATION (JUST ACTIVATE WHEN PLOTING VALIDATION FOR HORIZONTAL)
    minj1            = 1e-2 # epstest
    maxj1            = 5

    minj2            = 1e-1 # epstest
    maxj2            = 30

    # >>HORIZONTAL SANDERSE (D=0.078m)
    # x1,y1 = np.loadtxt('Sanderse2017_horizontal/well-posedness_sanderse.csv',
    #              unpack = True, 
    #              delimiter = ',')
    # x2,y2 = np.loadtxt('Sanderse2017_horizontal/globalstability_sanderse.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # # fig, ax = plt.subplots()

    # ax.loglog(x1,y1, 'k-', label = 'IKH Sanderse, Smith and Hendrix (2017)')
    # ax.loglog(x2,y2, 'k:', label = 'VKH Sanderse, Smith and Hendrix (2017)')

    # plt.ylim ((minj1, maxj1))
    # plt.xlim ((minj2, maxj2))
    # plt.grid (True, which = "both")
    # leg = ax.legend (loc = 'upper right',
    #                 fontsize = 14);
    # plt.rcParams ['figure.figsize'] = [12, 8]

    # matplotlib.rc ('xtick',
    #             labelsize = 18)     
    # matplotlib.rc ('ytick',
    #             labelsize = 18)
    # leg1  = ax.legend (loc = 'upper right',
    #                 shadow = True, 
    #                 frameon = True, 
    #                 fontsize = 14);

    # ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
    # ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
    # ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
    # ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

    # ax.set_xlabel(r'$u_{sg}$ [m/s]',
    #             fontsize = 18)
    # ax.set_ylabel(r'$u_{sl}$ [m/s]',
    #             fontsize = 18)

    #================================
    # >># Plot Barnea Shoham Taitel (1981) for validation 
    # https://automeris.io/WebPlotDigitizer/tutorial.html
    # horizontal pipe d = 0.051

    x1,y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_IKH.csv',
                    unpack = True, 
                    delimiter = ',')
    x2,y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_VKH.csv',
                    unpack = True, 
                    delimiter = ',')
    x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/shoham1982_0deg.csv',
                    unpack = True, 
                    delimiter = ',')
    x4,y4 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_hlD.csv',
                    unpack = True, 
                    delimiter = ',')

    # fig, ax = plt.subplots(1)

    area = area_scatter
    ax.loglog(x1,y1, 'ko-', alpha = alphascatter, label = 'IKH Barnea and Taitel (1994)')
    ax.loglog(x2,y2, 'kv-', alpha = alphascatter, label = 'VKH Barnea and Taitel (1994)')
    # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
    # ax.loglog(x3,y3, 'k^-', alpha = alphaplot, label = 'VKH Shoham (1982)')
    ax.loglog(x4,y4, 'ks-', alpha = alphascatter, label = r'$\alpha_l$ = 0.5')

    plt.ylim ((minj1, maxj1))
    plt.xlim ((minj2, maxj2))
    plt.grid (True, which = "both")
    leg = ax.legend (loc = 'best',
                    fontsize = 14);
    plt.rcParams ['figure.figsize'] = [12, 8]

    matplotlib.rc ('xtick',
                labelsize = 18)     
    matplotlib.rc ('ytick',
                labelsize = 18)

    handles,labels = ax.get_legend_handles_labels()

    handles = [handles[0], handles[1], handles[2], handles[3]]
    labels = [labels[0], labels[1], labels[2], labels[3]]

    leg1  = ax.legend (handles,
                    labels,
                    loc = 'best',
                    shadow = True, 
                    frameon = True, 
                    fontsize = 14);

    ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
    ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

    ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$', 
                fontsize = 18, 
                fontdict = font)
    ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$',
                fontdict = font,
                fontsize = 18)

    plt.text(0.15, 0.05, 'well-posed stable', size = 18)
    plt.text(0.5, 0.4, 'well-posed unstable', size = 18)
    plt.text(0.9, 3, 'ill-posed', size = 18)


    # =================
    # 0.25inc

    # x1,y1 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_IKH.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # x2,y2 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_VKH.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # x3,y3 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea 1994_025/shoham1982_025deg.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # x4,y4 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_hlD.csv',
    #                 unpack = True, 
    #                 delimiter = ',')

    # # fig, ax = plt.subplots(1)

    # area = 25
    # alphaplot = 0.6
    # ax.loglog(x1,y1, 'ko-', alpha = alphaplot, label = 'IKH Barnea and Taitel (1994)')
    # ax.loglog(x2,y2, 'kv-', alpha = alphaplot, label = 'VKH Barnea and Taitel (1994)')
    # # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
    # # ax.loglog(x3,y3, 'k^-', alpha = alphaplot, label = 'VKH Shoham (1982)')
    # # ax.loglog(x4,y4, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

    # plt.ylim ((minj1, maxj1))
    # plt.xlim ((minj2, maxj2))
    # plt.grid (True, which = "both")
    # leg = ax.legend (loc = 'best',
    #                 fontsize = 14);
    # plt.rcParams ['figure.figsize'] = [12, 8]

    # matplotlib.rc ('xtick',
    #             labelsize = 18)     
    # matplotlib.rc ('ytick',
    #             labelsize = 18)

    # handles,labels = ax.get_legend_handles_labels()

    # handles = [handles[0], handles[1], handles[2], handles[3]]
    # labels = [labels[0], labels[1], labels[2], labels[3]]

    # leg1  = ax.legend (handles,
    #                 labels,
    #                 loc = 'best',
    #                 shadow = True, 
    #                 frameon = True, 
    #                 fontsize = 14);

    # ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
    # ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
    # ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
    # ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

    # ax.set_xlabel(r'$u_{sg}$ [m/s]', 
    #             fontsize = 18, 
    #             fontdict = font)
    # ax.set_ylabel(r'$u_{sl}$ [m/s]',
    #             fontdict = font,
    #             fontsize = 18)

    # plt.text(0.3, 0.05, 'well-posed stable', size = 18)
    # plt.text(5.0, 0.02, 'well-posed unstable', size = 18)
    # plt.text(1.1, 2, 'ill-posed', size = 18)

    # =================
    # -5inc

    # x1,y1 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_IKH.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # # x2,y2 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_VKH.csv',
    # #                  unpack = True, 
    # #                  delimiter = ',')
    # x2,y2 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea1994-5/shoham1982_-5deg.csv',
    #                 unpack = True, 
    #                 delimiter = ',')
    # x3,y3 = np.loadtxt('barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_hlD.csv',
    #                 unpack = True, 
    #                 delimiter = ',')

    # # fig, ax = plt.subplots(1)

    # area = 25
    # alphaplot = 0.6
    # ax.loglog(x1,y1, 'ko-', alpha = alphaplot, label = 'IKH Barnea and Taitel (1994)')
    # # ax.loglog(x2,y2, 'kv-', alpha = alphaplot, label = 'VKH Barnea and Taitel (1994)')
    # # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
    # ax.loglog(x2,y2, 'k^-', alpha = alphaplot, label = 'VKH Shoham (1982)')
    # ax.loglog(x3,y3, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

    # plt.ylim ((minj1, maxj1))
    # plt.xlim ((minj2, maxj2))
    # plt.grid (True, which = "both")
    # leg = ax.legend (loc = 'best',
    #                 fontsize = 14);
    # plt.rcParams ['figure.figsize'] = [12, 8]

    # matplotlib.rc ('xtick',
    #             labelsize = 18)     
    # matplotlib.rc ('ytick',
    #             labelsize = 18)

    # handles,labels = ax.get_legend_handles_labels()

    # handles = [handles[0], handles[1], handles[2]]
    # labels = [labels[0], labels[1], labels[2]]

    # leg1  = ax.legend (handles,
    #                 labels,
    #                 loc = 'best',
    #                 shadow = True, 
    #                 frameon = True, 
    #                 fontsize = 14);

    # ax.xaxis.set_tick_params(which='major', size=10, direction='in', top='on')
    # ax.xaxis.set_tick_params(which='minor', size=7, direction='in', top='on')
    # ax.yaxis.set_tick_params(which='major', size=10, direction='in', right='on')
    # ax.yaxis.set_tick_params(which='minor', size=7, direction='in', right='on')

    # ax.set_xlabel(r'$u_{sg}$ [m/s]', 
    #             fontsize = 18, 
    #             fontdict = font)
    # ax.set_ylabel(r'$u_{sl}$ [m/s]',
    #             fontdict = font,
    #             fontsize = 18)

    # plt.text(3.0, 0.012, 'well-posed stable', size = 18)
    # plt.text(0.3, 0.5, 'well-posed unstable', size = 18)
    # plt.text(1.5, 3, 'ill-posed', size = 18)

#     plt.show ()
#     plt.savefig('figures/stability_map_2_'+ str(i) +'.pdf',
                # optimize = True,
                # transparent = True,  
                # dpi = 1000)

    i += 1